In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph

import pickle
import os

In [35]:
OPENAI_API_KEY = 'sk-XwwGmbuN8PZ2IX5CAkejMXIrPpsF4c435PEGWu0NIbT3BlbkFJAW1uxKGNBG1DyPZR43e06JGPrCvPrWKYmbDlTxzMoA'

In [43]:
NEO4J_URI="neo4j+s://9411be76.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="R8KvUc-AxBXH4I0ZMYYa0kwWwORNHQdg1sW-jLyQZZQ"

In [44]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [16]:

# Load the cleaned_documents list from the file
with open('final_pdf_pages.pkl', 'rb') as file:
    docs = pickle.load(file)

print("Documents loaded successfully!")


Documents loaded successfully!


In [17]:
# docs[0]

In [18]:
len(docs)

220

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separators=['\n'],
    chunk_size=2024,
    chunk_overlap=204,
    length_function=len,
)

In [20]:
data = text_splitter.split_documents(docs)

In [38]:
len(data)

615

In [24]:
# print(data[0].page_content)

In [25]:
type(data[0])

langchain_core.documents.base.Document

In [26]:
# # Saving Chunked data
# import pickle

# # Save the cleaned_documents list to a file
# with open('chunked_documents.pkl', 'wb') as file:
#     pickle.dump(cleaned_documents, file)

# print("Documents saved successfully!")

In [45]:
graph = Neo4jGraph()

In [46]:
llm = ChatOpenAI(model_name="gpt-4o")

In [47]:
llm_transformer = LLMGraphTransformer(llm=llm)

In [48]:
graph_documents = llm_transformer.convert_to_graph_documents(data)

In [ ]:
graph_documents

In [50]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [52]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [65]:
default_cypher = "MATCH (s)-[r:!CAUSE]->(t) RETURN s,r,t LIMIT 50"

In [66]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [68]:
# showGraph()